In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from specim_test.specim.specfuncs import spec2d
from astropy.io import fits as pf

In [2]:
file_loc = '../galaxy_spectra/DEIMOS_data/J1330_data/*'

In [3]:
file_list = []
out_file_det3 = []
out_file_det7 = []

for i,p in enumerate(glob.glob(file_loc)):
    if 'spec2d' in p and 'J1330' in p:
        file_list.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        out_file_det3.append('J1330_DEIMOS_spectra/J1330_deimos_det3_'+ frame + '_clean.fits')
        out_file_det7.append('J1330_DEIMOS_spectra/J1330_deimos_det7_'+ frame + '_clean.fits')

In [4]:
file_list = sorted(file_list)
file_list

['../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits',
 '../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0087-J1330m1_DEIMOS_2021Jun13T090838.746.fits',
 '../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0614_0085-J1330m1_DEIMOS_20210614T085223.635.fits']

In [5]:
out_file_det3 = sorted(out_file_det3)
out_file_det7 = sorted(out_file_det7)
out_file_det3, out_file_det7

(['J1330_DEIMOS_spectra/J1330_deimos_det3_d0613_0086_clean.fits',
  'J1330_DEIMOS_spectra/J1330_deimos_det3_d0613_0087_clean.fits',
  'J1330_DEIMOS_spectra/J1330_deimos_det3_d0614_0085_clean.fits'],
 ['J1330_DEIMOS_spectra/J1330_deimos_det7_d0613_0086_clean.fits',
  'J1330_DEIMOS_spectra/J1330_deimos_det7_d0613_0087_clean.fits',
  'J1330_DEIMOS_spectra/J1330_deimos_det7_d0614_0085_clean.fits'])

### finding out spatial location of 2d lens spectra in science image¶

In [6]:
file_list_spec1d = []
frame_list = []
for i,p in enumerate(glob.glob(file_loc)):
    if 'spec1d' in p and 'J1330' in p:
        file_list_spec1d.append(p)
        f = p.split('_')
        frame = f[-4]+ '_' + f[-3][:4]
        frame_list.append(frame)

In [7]:
file_list_spec1d

['../galaxy_spectra/DEIMOS_data/J1330_data/spec1d_d0614_0085-J1330m1_DEIMOS_20210614T085223.635.fits',
 '../galaxy_spectra/DEIMOS_data/J1330_data/spec1d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits',
 '../galaxy_spectra/DEIMOS_data/J1330_data/spec1d_d0613_0087-J1330m1_DEIMOS_2021Jun13T090838.746.fits']

In [8]:
for fl, frm in zip(file_list_spec1d, frame_list):
    sp = pf.open(fl)
    #m = sp.slitinfo['objid']=='lens'
    nspec = sp[0].header['NSPEC']
    print(frm, nspec)
    #if frm =='d0613_0105':
        #pass
    #elif frm =='d0613_0106':
        #pass
    #else:
    for i in range(nspec):
        if 'lens' in sp[i+1].header['HIERARCH MASKDEF_OBJNAME']:
            print(sp[i+1].header['NAME'])

d0614_0085 135
SPAT0311-SLIT0316-DET03
SPAT0311-SLIT0315-DET07
d0613_0086 135
SPAT0310-SLIT0316-DET03
SPAT0310-SLIT0315-DET07
d0613_0087 134
SPAT0310-SLIT0316-DET03
SPAT0310-SLIT0315-DET07


## median image after sky subtraction

In [ ]:
13_86 2,6 - [273, 363] ; 13_87 2,6 - [273, 363]; 14_85 2,6 - [273, 363]

In [12]:
xtrim_det = [273, 364]

spec_det_3 = []
spec_det_7 = []
for i, fl in enumerate(file_list):
    spec1 = spec2d.Spec2d(fl, hext=27, xtrim=xtrim_det)
    spec2 = spec2d.Spec2d(fl, hext=79, xtrim=xtrim_det)
    
    spec1.set_dispaxis('y')
    spec1.subtract_sky_2d(use_skymod=True)
    spec_det_3.append(spec1.data)
    
    spec2.set_dispaxis('y')
    spec2.subtract_sky_2d(use_skymod=True)
    spec_det_7.append(spec2.data)


Loading file ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits (HDU=27)
The input dataset was trimmed
 xrange: 273:365.  yrange: 0:4096
Final data dimensions (x y): 92 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 92


Loading file ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits (HDU=79)
The input dataset was trimmed
 xrange: 273:365.  yrange: 0:4096
Final data dimensions (x y): 92 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 92


Old value of dispaxis: x

Dispersion axis

In [13]:
median_im_det3 = np.median(np.asarray(spec_det_3), axis=0)
median_im_det7 = np.median(np.asarray(spec_det_7), axis=0)

In [14]:
pf.PrimaryHDU(median_im_det3).writeto('J1330_DEIMOS_spectra/J1330_median_image_det3.fits')
pf.PrimaryHDU(median_im_det7).writeto('J1330_DEIMOS_spectra/J1330_median_image_det7.fits')

### subtract sky and cosmic rays

In [18]:
extension = [27, 79]
trim = [10, 80]

median_im = ['J1330_DEIMOS_spectra/J1330_median_image_det3.fits', 
             'J1330_DEIMOS_spectra/J1330_median_image_det7.fits']

for fl, ofl1, ofl2 in zip(file_list, out_file_det3, out_file_det7):
        
    for ext, mdim in zip(extension, median_im):
        print(ext, mdim)
        spec = spec2d.Spec2d(fl, hext=ext, xtrim=xtrim_det)
        #spec2 = spec2d.Spec2d(fl, hext=66, xtrim=[970, 1080])
        spec.set_dispaxis('y')
        spec.subtract_sky_2d(use_skymod=True)
        spec.szap_new(infile=mdim)
        
        
        filled_data = spec.data[:, 11:80]
        mask = spec.szap_mask[:, 11:80]
        #print(filled_data.shape)
        ivar_im = pf.open(fl)[ext+1].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 11:80]
        sky_im = pf.open(fl)[ext+2].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 11:80]
        wav_im = pf.open(fl)[ext+7].data[spec.ymin:spec.ymax, spec.xmin:spec.xmax][:, 11:80]
            
        ivar_im[mask] = 1.e-5
        #print(ivar_im.shape)
        
        prim_hdr = pf.open(fl)[0].header
        prim_hdr.append(('OBJECT', 'J1330+1810'))
        prim_hdr.append(('', 'Trimmed and cleaned data by Pritom'))
        phdu = pf.PrimaryHDU(header=prim_hdr)
        
        sci_hdr = pf.Header()
        sci_hdr.append(('OBJECT', 'J1330+1810'))
        sci_hdr.append(('IMTYPE', 'SCIENCE_IMAGE'))
        sci_im_hdu = pf.ImageHDU(data=filled_data, header=sci_hdr, name='SCI_IM')

        ivar_hdr = pf.Header()
        ivar_hdr.append(('IMTYPE', 'IVAR_IMAGE'))
        ivar_im_hdu = pf.ImageHDU(data=ivar_im, header=ivar_hdr, name='IVAR')

        sky_hdr = pf.Header()
        sky_hdr.append(('IMTYPE', 'SKY_MODEL'))
        sky_im_hdu = pf.ImageHDU(data=sky_im, header=sky_hdr, name='SKY_MOD')

        wav_hdr = pf.Header()
        wav_hdr.append(('IMTYPE', 'WAV_IMAGE'))
        wav_im_hdu = pf.ImageHDU(data=wav_im, header=wav_hdr, name='WAV_IM')

        mask_hdr = pf.Header()
        mask_hdr.append(('IMTYPE', 'MASK'))
        cmt = 'pixels masked as cosmic rays and filled with pixel values at same pixel'\
            'location from median image'
        mask_hdr.append(('', cmt))
        mask_hdu = pf.ImageHDU(data=mask, header=mask_hdr, name='MASK')

        hdul = pf.HDUList([phdu, sci_im_hdu, ivar_im_hdu, sky_im_hdu, wav_im_hdu, mask_hdu ])
        
        if ext==27:
            hdul.writeto(ofl1)
            print(ofl1)
        else:
            hdul.writeto(ofl2)
            print(ofl2)

27 J1330_DEIMOS_spectra/J1330_median_image_det3.fits

Loading file ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits
-----------------------------------------------
Read in 2-dimensional spectrum from ../galaxy_spectra/DEIMOS_data/J1330_data/spec2d_d0613_0086-J1330m1_DEIMOS_2021Jun13T083648.787.fits (HDU=27)
The input dataset was trimmed
 xrange: 273:365.  yrange: 0:4096
Final data dimensions (x y): 92 x 4096

Dispersion axis:              x
N_pixels along dispersion axis: 92


Old value of dispaxis: x

Dispersion axis:              y
N_pixels along dispersion axis: 4096



pypeit generated sky model will be used for sky subtraction


From this point sky subtracted data will be used
 Spectrum Start:     0.00
 Spectrum End:     4095.00
 Dispersion (1st pixel):   1.00
 Dispersion (average):      1.00


From now cosmic ray rejected data will be used
J1330_DEIMOS_spectra/J1330_deimos_det3_d0613_0086_clean.fits
79 J1330_DEIMOS_spectra/J1330_